In [1]:
from features import ensembleFeature
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import torch
from torch import nn
import re
from sklearn.utils import shuffle
from utils import metricsCal
from torch.utils.data import DataLoader,TensorDataset
import math
import sys
import copy
import pickle
from torch.autograd import Variable
from sklearn.model_selection import KFold
import torch.nn.functional as F
import os
import Model
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import csv
from itertools import combinations,permutations
from sklearn.preprocessing import StandardScaler
from time import time
from thundersvm import SVC

%matplotlib inline

In [ ]:
type_name = ["FullTranscript","matureRNA"]
cell_name = ["A549","CD8T","Hek293_abacm","Hek293_sysy","HeLa","MOLM13"]
feature_name = ["binary","NCPA","emb","PSNP","ENAC","PseDNC","EIIP","PCP","DBPF"]

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")  #选择设备
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

Auc = np.zeros(len(type_name)*len(cell_name)*5*1023).reshape(1023,-1)
Acc = np.zeros(len(type_name)*len(cell_name)*5*1023).reshape(1023,-1)
Mcc = np.zeros(len(type_name)*len(cell_name)*5*1023).reshape(1023,-1)

col_name = []
row_name = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        for i in range(5):
            col_name.append(type_name[typei]+"_"+cell_name[celli]+"_"+str(i))
            train_data = pd.read_csv("Stacking_result/all_train_KFold/train_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,1:]
            test_data = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold"+str(i)+".csv").iloc[:,1:]
            
            train_data = np.array(train_data)
            test_data = np.array(test_data)
            
            x = list(range(10))
            fea_list = ['binary','NCPA','emb','PSNP','ENAC','PseDNC','EIIP','PCP','DBPF','Blastn']
            row = -1
            for j in range(10):
                for com_one in list(combinations(x, j+1)):
                    if typei==0 and celli==0 and i==0:
                        row_one = ""
                        for m in com_one:
                            row_one += fea_list[m]+"_"
                        row_one = row_one[0:-1]
                        row_name.append(row_one)
                    x_train = []
                    x_test = []
                    for m in com_one:
                        row_name += fea_list[m]+"_"
                        x_train.append(train_data[:,m:m+1])
                        x_test.append(test_data[:,m:m+1])
                    row+=1
                    model = SVC(kernel = "polynomial",gamma="auto", degree = 1, cache_size=7000)
                    clf = model.fit(np.array(x_train).reshape(len(com_one),-1).transpose(), train_data[:,-1])
                    th,_,_,_,_,_,_,acc,mcc,auc = metricsCal.get_train_metrics(clf.decision_function(np.array(x_train).reshape(len(com_one),-1).transpose()).reshape(-1),train_data[:,-1].reshape(-1))                
                    _,_,_,_,_,_,acc,mcc,auc = metricsCal.get_test_metrics(clf.decision_function(np.array(x_test).reshape(len(com_one),-1).transpose()).reshape(-1),test_data[:,-1].reshape(-1),th)
                    Auc[row][typei*30+celli*5+i]=auc
                    Acc[row][typei*30+celli*5+i]=acc
                    Mcc[row][typei*30+celli*5+i]=mcc

In [4]:
len(row_name)

1659903

In [7]:
np.max(np.mean(Auc,axis=1))

0.788137188646072

In [8]:
for i in range(len(np.mean(Auc,axis=1))):
    if np.mean(Auc,axis=1)[i] == np.max(np.mean(Auc,axis=1)):
        print(i)

1008


In [37]:
x = list(range(10))
row_name = []
fea_list = ['binary','NCPA','emb','PSNP','ENAC','PseDNC','EIIP','PCP','DBPF','Blastn']
for j in range(10):
    for com_one in list(combinations(x, j+1)):
        row_one = ""
        for m in com_one:
            row_one += fea_list[m]+"_"
        row_one = row_one[0:-1]
        row_name.append(row_one)

In [19]:
row_name[1008]

'NCPA_emb_PSNP_PseDNC_EIIP_PCP_DBPF_Blastn'

In [41]:
#添加第一列
Auc_col = np.append(np.array(row_name).reshape(-1,1),Auc,axis=1)
#补全header
col_kk = col_name
col_kk.insert(0,"row_name")
#保存文件
pd.DataFrame(Auc_col).to_csv("Stacking_result/Poly_10.csv",header = col_kk,index=0)
#读取文件
data = pd.read_csv("Stacking_result/Poly_10.csv",index_col = "row_name")
for i in range(len(np.array(data.index))):
    if np.array(data.index)[i]=="NCPA_emb_PSNP_PCP_Blastn":
        print(i)

524


In [53]:
da = 520
print(np.mean(np.array(data),axis=1)[da])
data.index[da]

0.7803698767530615


'NCPA_emb_PSNP_EIIP_PCP'

In [26]:
np.max(np.mean(np.array(data),axis=1))

0.788137188646072

In [27]:
np.mean(np.array(data),axis=1)[524]

0.7877487439576926

In [95]:
data = pd.read_csv("Stacking_result/Poly_10_Standard.csv",index_col = "row_name")

In [38]:
col_name = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        for i in range(5):
            col_name.append(type_name[typei]+"_"+cell_name[celli]+"_"+str(i))

In [96]:
np.max(np.mean(np.array(data),axis=1))

0.788137306158226

In [97]:
print("一种特征的")
print(np.max(  np.mean(np.array(data),axis=1)[0:9]  ))
print("两种特征的")
print(np.max(  np.mean(np.array(data),axis=1)[10:54]  ))
print("三种特征的")
print(np.max(  np.mean(np.array(data),axis=1)[55:174]  ))
print("四种特征的")
print(np.max(  np.mean(np.array(data),axis=1)[175:384]  ))
print("五种特征的")
print(np.max(  np.mean(np.array(data),axis=1)[385:636]  ))
print("六种特征的")
print(np.max(  np.mean(np.array(data),axis=1)[637:846]  ))
print("七种特征的")
print(np.max(  np.mean(np.array(data),axis=1)[847:966]  ))
print("八种特征的")
print(np.max(  np.mean(np.array(data),axis=1)[967:1011]  ))
print("九种特征的")
print(np.max(  np.mean(np.array(data),axis=1)[1012:1021]  ))
print("十种特征的")
print(np.max(  np.mean(np.array(data),axis=1)[1021:1022]  ))

一种特征的
0.765269909723055
两种特征的
0.7793552200575783
三种特征的
0.7840974307177527
四种特征的
0.786642817182932
五种特征的
0.7878195384048562
六种特征的
0.7880296173030603
七种特征的
0.7880897566218974
八种特征的
0.788137306158226
九种特征的
0.7880669411073417
十种特征的
0.787972342452658


In [116]:
best_auc = 0
best_row = ""
for i in range(len(data)):
    if row_name[i].count("_")==9:#and row_name[i].count("Blastn")==0:
        if np.mean(np.array(data)[i])>best_auc:
            best_auc = np.mean(np.array(data)[i])
            best_row = row_name[i]
print(best_auc)
print(best_row)

0.7879072573098804
binary_NCPA_emb_PSNP_ENAC_PseDNC_EIIP_PCP_DBPF_Blastn


In [ ]:
#标准化的
0.7652699097230546
DBPF

0.7794928740301417
DBPF_Blastn

0.7840974307177525
emb_DBPF_Blastn

0.7866428171829319
NCPA_emb_PCP_Blastn

0.7878195384048566
emb_PSNP_PCP_DBPF_Blastn

0.7880296173030603
NCPA_emb_PSNP_PCP_DBPF_Blastn

0.7880897566218973
NCPA_emb_PSNP_PseDNC_PCP_DBPF_Blastn

0.7881373061582261
NCPA_emb_PSNP_PseDNC_EIIP_PCP_DBPF_Blastn

0.7880669411073417
binary_NCPA_emb_PSNP_PseDNC_EIIP_PCP_DBPF_Blastn

0.7879072573098804
binary_NCPA_emb_PSNP_ENAC_PseDNC_EIIP_PCP_DBPF_Blastn

In [ ]:
#标准化的
0.7652699097230546
DBPF

0.7736447930649347
PSNP_PCP

0.7787071005025502
emb_PSNP_PCP

0.7802217912025209
emb_PSNP_PCP_DBPF

0.7804871122767592
NCPA_emb_PSNP_PCP_DBPF

0.7806260136004176
NCPA_emb_PSNP_EIIP_PCP_DBPF

0.7807561310241518
NCPA_emb_PSNP_PseDNC_EIIP_PCP_DBPF

0.7807359719717136
binary_NCPA_emb_PSNP_PseDNC_EIIP_PCP_DBPF

0.7807173900770106
binary_NCPA_emb_PSNP_ENAC_PseDNC_EIIP_PCP_DBPF

In [69]:
0.7652699183378325
DBPF

0.7794933773296329
DBPF_Blastn

0.7840968989154801
emb_DBPF_Blastn

0.7866418288591108
NCPA_emb_PCP_Blastn

0.7878197502353922
emb_PSNP_PCP_DBPF_Blastn

0.7880312888993214
NCPA_emb_PSNP_PCP_DBPF_Blastn

0.7880911089240393
NCPA_emb_PSNP_PseDNC_PCP_DBPF_Blastn

0.788137188646072
NCPA_emb_PSNP_PseDNC_EIIP_PCP_DBPF_Blastn

0.7880691260432905
binary_NCPA_emb_PSNP_PseDNC_EIIP_PCP_DBPF_Blastn

0.7879064098122479
binary_NCPA_emb_PSNP_ENAC_PseDNC_EIIP_PCP_DBPF_Blastn

9

In [86]:
0.7652699183378325
DBPF

0.7736459253879352
PSNP_PCP

0.7787062792712284
emb_PSNP_PCP

0.7802190128673808
emb_PSNP_PCP_DBPF

0.7804866415541133
NCPA_emb_PSNP_PCP_DBPF

0.780624968074786
NCPA_emb_PSNP_EIIP_PCP_DBPF

0.7807561955074988
NCPA_emb_PSNP_PseDNC_EIIP_PCP_DBPF

0.7807351183573215
binary_NCPA_emb_PSNP_PseDNC_EIIP_PCP_DBPF

0.7807172487578738
binary_NCPA_emb_PSNP_ENAC_PseDNC_EIIP_PCP_DBPF

0.7652699183378325